# ALL imports

In [1]:
import pandas as pd
import numpy as np
import joblib
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

# Loading the data

In [2]:
df=pd.read_csv("Health.csv")
df.head()

,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,panic disorder,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,panic disorder,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,panic disorder,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,panic disorder,1,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,panic disorder,1,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.shape

(246945, 378)

In [4]:
# Removing where disease apper less than 1 times
counts = df['diseases'].value_counts()
df=df[df['diseases'].isin(counts[counts > 1].index)]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246926 entries, 0 to 246944
Columns: 378 entries, diseases to neck weakness
dtypes: int64(377), object(1)
memory usage: 714.0+ MB


In [6]:
df.isna().sum().sum()

np.int64(0)

# Splititng and Encoding

In [7]:
X=df.drop("diseases",axis=1).values
y=df["diseases"].values

In [8]:
encoder=LabelEncoder()
y=encoder.fit_transform(y)

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42,shuffle=True,stratify=y)
features=list(df.columns)
features.remove("diseases")

In [10]:
# Saving encoder classes
with open("encoder_class.pkl", "wb") as f:
    pickle.dump(encoder.classes_, f)

In [11]:
# Saving columns
with open("symbtoms.pkl", "wb") as f:
    pickle.dump(df.columns, f)

# Model Making

In [12]:
model=keras.Sequential()
model.add(keras.layers.Dense(1024,activation="relu",input_dim=X_train.shape[1]))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(512,activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(256,activation="relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dense(754,activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

model.summary()

C:\Users\User\Desktop\NN\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1024)                │         387,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 754)                 │          97,266 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,192,434 (4.55 MB)

 Trainable params: 1,191,154 (4.54 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [13]:
callbacks=tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                          patience=10,
                                          min_delta=0.0001,
                                          verbose=True)

In [14]:
history=model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),callbacks=callbacks)

Epoch 1/50
6945/6945 ━━━━━━━━━━━━━━━━━━━━ 164s 23ms/step - accuracy: 0.6371 - loss: 1.6439 - val_accuracy: 0.8339 - val_loss: 0.4978
Epoch 2/50
6945/6945 ━━━━━━━━━━━━━━━━━━━━ 132s 19ms/step - accuracy: 0.8204 - loss: 0.5621 - val_accuracy: 0.8436 - val_loss: 0.4389
Epoch 3/50
6945/6945 ━━━━━━━━━━━━━━━━━━━━ 125s 18ms/step - accuracy: 0.8346 - loss: 0.4908 - val_accuracy: 0.8484 - val_loss: 0.4147
Epoch 4/50
6945/6945 ━━━━━━━━━━━━━━━━━━━━ 131s 19ms/step - accuracy: 0.8441 - loss: 0.4485 - val_accuracy: 0.8502 - val_loss: 0.3978
Epoch 5/50
6945/6945 ━━━━━━━━━━━━━━━━━━━━ 131s 19ms/step - accuracy: 0.8494 - loss: 0.4187 - val_accuracy: 0.8502 - val_loss: 0.3849
Epoch 6/50
6945/6945 ━━━━━━━━━━━━━━━━━━━━ 135s 19ms/step - accuracy: 0.8527 - loss: 0.4025 - val_accuracy: 0.8526 - val_loss: 0.3759
Epoch 7/50
6945/6945 ━━━━━━━━━━━━━━━━━━━━ 134s 19ms/step - accuracy: 0.8547 - loss: 0.3873 - val_accuracy: 0.8562 - val_loss: 0.3647
Epoch 8/50
6945/6945 ━━━━━━━━━━━━━━━━━━━━ 138s 20ms/step - accuracy: 

In [15]:
model.save("predictor.keras")

# Completed